In [1]:
## CHARGEMENT DES BIBLIOTHEQUES
############################################################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import xgboost as xgb
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from umap import UMAP
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
## FONCTIONS UTILES
############################################################

##
# CHARGEMENT DE FICHIERS JSON

# Fonction qui permet de charger un fichier json dans un dictionnaire
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data


##
# CHARGEMENT DU DATAFRAME

# On charge les données dans un dataframe "df"
def load_dataset(path):
    df = pd.read_csv(path)

    return df


##
# FILTRAGE DE COLONNES

# On récupère ici seulement les évènements qui nous intéressent, ceux liés à des violences politiques
# On conserve 'Strategic developments' car elle permet de capturer le contexte
def columns_filtering(df):
    events_filter = ['Battles', 'Explosions/Remote violence',
                    'Violence against civilians', 'Strategic developments']

    df = df[df['event_type'].isin(events_filter)]
    
    return df


##
# CREATION DE NOUVELLES COLONNES

def add_new_colums_dates(df):
    ##
    # On transforme la colonne "event_date" en datetime pour la manipuler plus facilement
    df['event_date'] = pd.to_datetime(df['event_date'])

    # On crée la colonne "month" en utilisant l'attribut month de datetime
    df['month'] = df['event_date'].dt.month
    # On ajoute la colonne juste après "year"
    df.insert(3, 'month', df.pop('month'))

    # On crée la colonne "day" en utilisant l'attribut day de datetime
    df['day'] = df['event_date'].dt.day
    # On ajoute la colonne juste après "month"
    df.insert(5, 'day', df.pop('day'))
    
    return df


def add_new_colums_actors(df):
    ##
    # On charge les données du fichier json "actor_type" dans un dictionnaire
    # qui associe les valeurs des colonnes "inter1" et "inter2"
    # au nom de chaque catégorie d'acteur (source : codebook ACLED)
    actor_type = load_json_file("../references/actor_type.json")

    # On convertit les clés du dictionnaire en entiers
    actor_type = {int(k): v for k, v in actor_type.items()}

    # On ajoute les colonnes "actor1_type" et "actor2_type" au dataframe
    df['actor1_type'] = df['inter1'].map(actor_type)
    df['actor2_type'] = df['inter2'].map(actor_type)

    return df


def add_new_colums_interaction(df):
    ##
    # On charge les données du fichier json "interaction_type" dans un dictionnaire
    # associant aux valeurs de la colonne "interaction"
    # les 2 acteurs impliqués dans une confrontation (source : codebook ACLED)
    interaction_type = load_json_file("../references/interaction_type.json")

    # On convertit les clés du dictionnaire en entiers
    interaction_type = {int(k): v for k, v in interaction_type.items()}

    # On ajoute une colonne "interaction_type" au dataframe
    df['interaction_type'] = df['interaction'].map(interaction_type)

    return df


def add_new_colums_terrorist_group(df):
    # On crée une nouvelle colonne qui indique si pour un évènement l'un des
    # acteurs est une organisation terroriste
    # On charge les données du fichier json "terrorist_group_filiation"
    # dans un dictionnaire associant organisation terroriste et organisation mère
    terrorist_group_filiation = load_json_file("../references/terrorist_group_filiation.json")

    # On crée une liste contenant les organisations terroristes à partir des indices de ce dictionnaire
    terrorist_groups = list(terrorist_group_filiation.keys())

    # On crée une fonction pour vérifier si un acteur est une organisation terroriste
    def is_terrorist_actor(actor):
        return actor in terrorist_groups

    # On ajoute la colonne "is_terrorist_group_related" au dataframe
    df['is_terrorist_group_related'] = (df['actor1'].apply(is_terrorist_actor) |
                                df['assoc_actor_1'].apply(is_terrorist_actor) |
                                df['actor2'].apply(is_terrorist_actor) |
                                df['assoc_actor_2'].apply(is_terrorist_actor)).astype(int)

    # On crée une fonction de mapping pour associer les valeurs du dictionnaire aux acteurs
    # S'il n'y a pas de valeur on retourne "None" car cela veut simplement dire que l'évènement
    # n'est pas lié à une organisation terroriste et qu'il n'y a donc pas de lien de filiation
    # avec une organisation mère
    def map_filiation(row):
        for actor in ['actor1', 'assoc_actor_1', 'actor2', 'assoc_actor_2']:
            if row[actor] in terrorist_group_filiation:
                return terrorist_group_filiation[row[actor]]
        return "None"

    # On ajoute la colonne "terrorist_group_filiation" au dataframe
    df['terrorist_group_filiation'] = df.apply(map_filiation, axis=1)

    return df


def add_new_colums_pmc_group(df):
    # On crée une nouvelle colonne qui indique si pour un évènement l'un
    # des acteurs est une pmc russe
    pmc_groups = ['Wagner Group']

    # On crée une fonction pour vérifier si un acteur est une pmc russe
    def is_pmc_actor(actor):
        return actor in pmc_groups

    # On ajoute la colonne "is_pmc_related" au dataframe
    df['is_pmc_related'] = (df['actor1'].apply(is_pmc_actor) |
                                df['assoc_actor_1'].apply(is_pmc_actor) |
                                df['actor2'].apply(is_pmc_actor) |
                                df['assoc_actor_2'].apply(is_pmc_actor)).astype(int)
    
    return df


##
# SUPPRESSION DE COLONNES

def delete_columns(df):
    # On supprime les colonnes qui ne nous serons d'aucune utilité
    columns_to_drop = [
        'time_precision',
        'disorder_type',
        'sub_event_type',
        'actor1',
        'assoc_actor_1',
        'actor2',
        'assoc_actor_2',
        'inter1',
        'inter2',
        'interaction',
        'admin2',
        'admin3',
        'iso',
        'region',
        'location',
        'latitude',
        'longitude',
        'geo_precision',
        'source',
        'source_scale',
        'notes',
        'tags',
        'timestamp',
        'civilian_targeting',
        'event_id_cnty'
    ]

    df = df.drop(columns=columns_to_drop, axis=1)

    return df


##
# GESTION DES VALEURS MANQUANTES

def fill_nan_values(df):
    # Pour la colonne "actor2_type" on va remplacer les valeurs manquantes par "None" car cela
    # indique juste que dans certains cas qu'il y a un seul acteur et pas un manque de valeur
    df['actor2_type'] = df['actor2_type'].fillna("None")

    return df


##
# Fonction pour mapper les noms de sous-régions aux identifiants de coordonnées
def map_regions_to_ids(data, mapping):
    data['region_id'] = data['admin1'].map(mapping)
    return data

In [3]:
## CONSTRUCTION DU DATAFRAME HISTORIQUE "past_data"
############################################################

def build_past_data(df):
    # On compte les occurences de chaque valeur des colonnes 'actor1_type' et 'actor2_type'
    actor_counts = df.melt(id_vars=['event_date', 'country', 'admin1', 'year', 'month', 'is_terrorist_group_related', 'fatalities', 'event_type'], 
                        value_vars=['actor1_type', 'actor2_type'], 
                        var_name='actor_role', value_name='actor')

    actor_counts = actor_counts.groupby(['country', 'admin1', 'year', 'month', 'actor']).size().unstack(fill_value=0).reset_index()

    # On compte les occurences de chaque type d'évènement
    event_type_counts = df.pivot_table(index=['country', 'admin1', 'year', 'month'], 
                                    columns='event_type', 
                                    aggfunc='size', 
                                    fill_value=0).reset_index()

    # On agrège les données par mois, par pays, par sous-région
    past_data = df.groupby(['country', 'admin1', 'year', 'month']).agg(
        total_events=('event_date', 'count'),
        terrorist_events=('is_terrorist_group_related', 'sum'),
        fatalities=('fatalities', 'sum')
    ).reset_index()

    # On fusionne les variables des acteurs et des types d'évènements avec les données agrégées
    past_data = past_data.merge(actor_counts, on=['country', 'admin1', 'year', 'month'], how='left')
    past_data = past_data.merge(event_type_counts, on=['country', 'admin1', 'year', 'month'], how='left')

    # On supprime les colonnes 'None' et 'Protesters' qui concernent les acteurs mais sont sans intérêt pour notre étude
    # ou bien la colonne 'Rebel Groups' qui est dérivée de la variable cible
    past_data = past_data.drop(columns=['None', 'Protesters', 'Rebel Groups'], axis=1)

    # On crée des features décalées (lags) pour capturer les tendances passées des événements totaux et des événements terroristes
    for admin in past_data['admin1'].unique():  # Pour chaque valeur unique dans la colonne 'admin1'
        
        for lag in range(1, 7):  # Pour chaque décalage (lag) de 1 à 6
            lag_total_events = past_data[past_data['admin1'] == admin]['total_events'].shift(lag)  # Crée une série décalée pour 'total_events' pour l'admin actuel
            lag_terrorist_events = past_data[past_data['admin1'] == admin]['terrorist_events'].shift(lag)  # Crée une série décalée pour 'terrorist_events' sans filtrer par admin
            past_data.loc[past_data['admin1'] == admin, f'total_events_lag_{lag}'] = lag_total_events  # Ajoute la série décalée pour 'total_events' dans une nouvelle colonne pour l'admin actuel
            past_data.loc[past_data['admin1'] == admin, f'terrorist_events_lag_{lag}'] = lag_terrorist_events  # Ajoute la série décalée pour 'terrorist_events' dans une nouvelle colonne pour l'admin actuel
            past_data[[f'total_events_lag_{lag}', f'terrorist_events_lag_{lag}']] = past_data[
                [f'total_events_lag_{lag}', f'terrorist_events_lag_{lag}']
                ].fillna(0)  # Remplit les valeurs manquantes dans les colonnes décalées avec des zéros

    # On supprime les valeurs NA générées par les lags
    # past_data.dropna(inplace=True)  # Optionnel : supprime les lignes contenant des valeurs NA (commenté ici)

    # On supprime la colonne "total_events" qui est trop corrélée à notre variable cible
    past_data = past_data.drop(columns=['total_events'], axis=1)

    return past_data


In [4]:
## TRANSFORMATION DES DONNEES DU DATAFRAME "past_data"
############################################################

def past_data_transformation(df):

    ##
    # On trie notre dataframe par année puis mois dans l'ordre croissant
    # pour que les données de test concernent les événements les plus récents
    df = df.sort_values(by=['year', 'month'])

    # Séparation des variables explicatives de notre variable cible
    X = df.drop(columns='terrorist_events', axis=1)
    y = df['terrorist_events']

    # Division en ensembles d'entraînement et de test avec shuffle à False
    # pour que les données de test concernent les événements les plus récents (6 derniers mois)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # On sauvegarde une version non encodée de X_test
    X_test_no_encoding = X_test


    ##
    # On standardise les variables numériques exceptées les variables catégorielles
    # ainsi que l'année et le mois qui subiront une autre transformation
    cat_columns=['country', 'admin1']
    X_train_numerical_vars = [col for col in X_train.columns if col not in cat_columns + ['year', 'month']]
    X_test_numerical_vars = [col for col in X_test.columns if col not in cat_columns + ['year', 'month']]
    # Extraction des données numériques
    X_train_numerical = X_train[X_train_numerical_vars].values
    X_test_numerical = X_test[X_test_numerical_vars].values

    # Initialisation d'un scaler
    scaler = StandardScaler()
    X_train[X_train_numerical_vars] = scaler.fit_transform( X_train_numerical)
    X_test[X_test_numerical_vars] = scaler.transform(X_test_numerical)


    ##
    # Hot One Encoding des variables catégorielles
    X_train_encoded = pd.get_dummies(X_train, columns=cat_columns, dtype=int)
    X_test_encoded = pd.get_dummies(X_test, columns=cat_columns, dtype=int)
    # On aligne les colonnes de X_test_encoded sur celles de X_train_encoded
    # pour s'assurer qu'ils ont les mêmes colonnes
    X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)


    ##
    # Transformation trigonométrique de 'year' et 'month' pour la saisonnalité
    X_train_encoded['month_sin'] = np.sin(2 * np.pi * X_train_encoded['month'] / 12)
    X_train_encoded['month_cos'] = np.cos(2 * np.pi * X_train_encoded['month'] / 12)
    X_train_encoded['year_sin'] = np.sin(2 * np.pi * (X_train_encoded['year'] - X_train_encoded['year'].min()) / 
                                    (X_train_encoded['year'].max() - X_train_encoded['year'].min() + 1))
    X_train_encoded['year_cos'] = np.cos(2 * np.pi * (X_train_encoded['year'] - X_train_encoded['year'].min()) / 
                                    (X_train_encoded['year'].max() - X_train_encoded['year'].min() + 1))
    
    X_test_encoded['month_sin'] = np.sin(2 * np.pi * X_test_encoded['month'] / 12)
    X_test_encoded['month_cos'] = np.cos(2 * np.pi * X_test_encoded['month'] / 12)
    X_test_encoded['year_sin'] = np.sin(2 * np.pi * (X_test_encoded['year'] - X_test_encoded['year'].min()) / 
                                    (X_test_encoded['year'].max() - X_test_encoded['year'].min() + 1))
    X_test_encoded['year_cos'] = np.cos(2 * np.pi * (X_test_encoded['year'] - X_test_encoded['year'].min()) / 
                                    (X_test_encoded['year'].max() - X_test_encoded['year'].min() + 1))

    # Suppression des colonnes 'month' et 'year' après transformation
    X_train_encoded = X_train_encoded.drop(columns=['month', 'year'])
    X_test_encoded = X_test_encoded.drop(columns=['month', 'year'])

    return X_train_encoded, X_test_encoded, X_test_no_encoding, y_train, y_test, scaler

In [5]:
## CONSTRUCTION DU DATAFRAME FUTUR "future_data"
## POUR LES PREDICTIONS DES 6 PROCHAINS MOIS
############################################################

def build_future_data(past_data):
    unique_country_admin1 = past_data[['country', 'admin1']].drop_duplicates()

    # On souhaite d'abord obtenir le dernier mois de la dernière l'année dans past_data
    last_year = past_data['year'].max()
    last_month = past_data[past_data['year'] == last_year]['month'].max()

    # On crée ensuite une liste de dictionnaires pour les 6 prochains mois avec les colonnes explicatives vides
    future_data_list = []

    # On ajoute les 6 prochains mois à future_data_list avec les colonnes connues "country" et "admin1" remplies
    for i in range(1, 7):
        next_month = last_month + i
        next_year = last_year
        if next_month > 12:
            next_month -= 12
            next_year += 1
        for _, row in unique_country_admin1.iterrows():
            future_data_list.append({
                'year': next_year, 
                'month': next_month, 
                'country': row['country'], 
                'admin1': row['admin1']
            })

    # On crée notre dataframe "future_data" à partir de la liste de dictionnaires
    future_data = pd.DataFrame(future_data_list, columns=past_data.columns)

    # On remplit les autres colonnes explicatives avec des valeurs manquantes
    for col in past_data.columns:
        if col not in ['year', 'month', 'country', 'admin1']:
            future_data[col] = np.nan

    # Liste des colonnes pour lesquelles on veut remplir les valeurs manquantes
    # columns_to_fill = [
    #     'total_events', 'fatalities', 'Civilians', 'External/Other Forces',
    #     'Identity Militias', 'Political Militias',
    #     'State Forces', 'Battles', 'Explosions/Remote violence',
    #     'Strategic developments', 'Violence against civilians',
    #     'total_events_lag_1', 'terrorist_events_lag_1',
    #     'total_events_lag_2', 'terrorist_events_lag_2',
    #     'total_events_lag_3', 'terrorist_events_lag_3',
    #     'total_events_lag_4', 'terrorist_events_lag_4',
    #     'total_events_lag_5', 'terrorist_events_lag_5',
    #     'total_events_lag_6', 'terrorist_events_lag_6'
    # ]

    columns_to_fill = [
        'fatalities', 'Civilians', 'External/Other Forces',
        'Identity Militias', 'Political Militias',
        'State Forces', 'Battles', 'Explosions/Remote violence',
        'Strategic developments', 'Violence against civilians',
        'total_events_lag_1', 'terrorist_events_lag_1',
        'total_events_lag_2', 'terrorist_events_lag_2',
        'total_events_lag_3', 'terrorist_events_lag_3',
        'total_events_lag_4', 'terrorist_events_lag_4',
        'total_events_lag_5', 'terrorist_events_lag_5',
        'total_events_lag_6', 'terrorist_events_lag_6'
    ]


    # On groupe les données par 'country', 'admin1', 'month' pour calculer les moyennes dans past_data
    grouped_data = past_data.groupby(['country', 'admin1', 'month']).mean().reset_index()

    # On remplace les valeurs manquantes dans future_data par les moyennes correspondantes de past_data
    # en fonction du mois. Ex : si on veut calculer les valeurs des variables du mois de juillet 2024,
    # on va calculer la moyenne des variables des mois de juillet des années précédentes pour avoir des
    # valeurs de test réalistes.
    for index, row in future_data.iterrows():
        country = row['country']
        admin1 = row['admin1']
        month = row['month']
        for var in columns_to_fill:
            mean_value = grouped_data[(grouped_data['country'] == country) & 
                                    (grouped_data['admin1'] == admin1) & 
                                    (grouped_data['month'] == month)][var].values
            if len(mean_value) > 0:
                future_data.at[index, var] = mean_value[0]

    # Certaines valeurs sont manquantes car les combinaisons (country, admin1, month) n'existent pas
    # on va donc créer un dataframe contenant les combinaisons uniques de (country, admin1, month) dans future_data
    future_combinations = future_data[['country', 'admin1', 'month']].drop_duplicates()

    # Puis un dataframe contenant les combinaisons uniques de (country, admin1, month) dans past_data
    monthly_combinations = past_data[['country', 'admin1', 'month']].drop_duplicates()

    # Ensuite on recherche les combinaisons présentes dans future_data mais absentes dans past_data
    missing_combinations = future_combinations[~future_combinations.isin(monthly_combinations)].dropna()

    # Affichage des combinaisons manquantes
    # print("Combinaisons manquantes dans past_data :\n"missing_combinations)

    # En fonction de ces combinaisons, on vient remplir les valeurs manquantes dans future_data
    # à partir du calcul de la moyenne des autres mois pour la combinaison (country, admin1)
    for index, row in missing_combinations.iterrows():
        # Sélection des lignes correspondant à la combinaison de country et admin1 dans past_data
        matching_rows = past_data[(past_data['country'] == row['country']) & (past_data['admin1'] == row['admin1'])]
        # Calcul de la moyenne des valeurs des variables pour tous les mois existants
        mean_values = matching_rows[columns_to_fill].mean()
        # Remplissage des valeurs manquantes dans future_data avec la moyenne calculée
        future_data.loc[(future_data['country'] == row['country']) & (future_data['admin1'] == row['admin1']) & (future_data['month'] == row['month']), columns_to_fill] = mean_values.values

    # Enfin on remplace les NaN des lignes restantes par la valeur la plus fréquente dans chaque colonne
    future_data = future_data.fillna(future_data.mode().iloc[0])

    # On supprime la colonne de variable cible et on vérifie qu'il n'y a plus de NaN
    future_data = future_data.drop(columns=['terrorist_events'])
    future_data.info()

    # On arrondit les valeurs calculées pour les variables explicatives afin d'obtenir des valeurs entières
    future_data = future_data.round(0)

    return future_data

In [6]:
## TRANSFORMATION DES DONNEES DU DATAFRAME "future_data"
############################################################

def future_data_transformation(df, scaler):
    ##
    # Préparation du dataframe "future_data" pour le modèle de ML
    future_data_encoded = df.copy()

    # On standardise les variables numériques exceptées les variables catégorielles
    # ainsi que l'année et le mois qui subiront une autre transformation
    cat_columns=['country', 'admin1']
    future_data_numerical_vars = [col for col in future_data_encoded.columns if col not in cat_columns + ['year', 'month']]
    # Extraction des données numériques
    future_data_numerical = df[future_data_numerical_vars].values

    # On utilise le scaler ajusté sur les données d'entraînement pour transformer les données du dataframe
    future_data_encoded[future_data_numerical_vars] = scaler.transform(future_data_numerical)

    # Hot One Encoding des variables catégorielles
    future_data_encoded = pd.get_dummies(future_data_encoded, columns=cat_columns, dtype=int)

    # Transformation trigonométrique de 'year' et 'month' pour la saisonnalité
    future_data_encoded['month_sin'] = np.sin(2 * np.pi * future_data_encoded['month'] / 12)
    future_data_encoded['month_cos'] = np.cos(2 * np.pi * future_data_encoded['month'] / 12)
    future_data_encoded['year_sin'] = np.sin(2 * np.pi * (future_data_encoded['year'] - future_data_encoded['year'].min()) / 
                                    (future_data_encoded['year'].max() - future_data_encoded['year'].min() + 1))
    future_data_encoded['year_cos'] = np.cos(2 * np.pi * (future_data_encoded['year'] - future_data_encoded['year'].min()) / 
                                    (future_data_encoded['year'].max() - future_data_encoded['year'].min() + 1))

    # Suppression des colonnes 'month' et 'year' après transformation
    future_data_encoded = future_data_encoded.drop(columns=['month', 'year'])

    return future_data_encoded

In [7]:
## CREATION DES DATAFRAMES HISTORIQUES ET FUTURS
############################################################

# On applique les différentes étapes de préparation à notre dataframe initial
df = load_dataset('../data/raw/terrorisme_sahel.csv')
df = columns_filtering(df)
df = add_new_colums_dates(df)
df = add_new_colums_actors(df)
df = add_new_colums_interaction(df)
df = add_new_colums_terrorist_group(df)
df = add_new_colums_pmc_group(df)
df = delete_columns(df)
df = fill_nan_values(df)

# On crée notre dataframe qui sera utilisé pour entrainer notre modèle
# sur les données historiques
past_data = build_past_data(df)

# On crée notre dataframe qui sera utilisé pour effectuer les prédictions
# sur les 6 prochains mois
future_data = build_future_data(past_data)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   country                     186 non-null    object 
 1   admin1                      186 non-null    object 
 2   year                        186 non-null    int64  
 3   month                       186 non-null    int64  
 4   fatalities                  186 non-null    float64
 5   Civilians                   186 non-null    float64
 6   External/Other Forces       186 non-null    float64
 7   Identity Militias           186 non-null    float64
 8   Political Militias          186 non-null    float64
 9   State Forces                186 non-null    float64
 10  Battles                     186 non-null    float64
 11  Explosions/Remote violence  186 non-null    float64
 12  Strategic developments      186 non-null    float64
 13  Violence against civilians  186 non

In [ ]:
## ETUDE DE LINEARITE
############################################################
    

def encode_and_transform(df):
    df_encoded = df.copy()
    # On standardise les variables numériques exceptées les variables catégorielles
    # ainsi que l'année et le mois qui subiront une autre transformation
    cat_columns=['country', 'admin1']
    df_encoded_numerical_vars = [col for col in df_encoded.columns if col not in cat_columns + ['year', 'month']]
    # Extraction des données numériques
    df_encoded_numerical = df_encoded[df_encoded_numerical_vars].values

    # Initialisation d'un scaler
    scaler = StandardScaler()
    df_encoded[df_encoded_numerical_vars] = scaler.fit_transform(df_encoded_numerical)

    # Hot One Encoding des variables catégorielles
    df_encoded = pd.get_dummies(df_encoded, columns=cat_columns, dtype=int)

    # Transformation trigonométrique de 'year' et 'month' pour la saisonnalité
    df_encoded['month_sin'] = np.sin(2 * np.pi * df_encoded['month'] / 12)
    df_encoded['month_cos'] = np.cos(2 * np.pi * df_encoded['month'] / 12)
    df_encoded['year_sin'] = np.sin(2 * np.pi * (df_encoded['year'] - df_encoded['year'].min()) / 
                                    (df_encoded['year'].max() - df_encoded['year'].min() + 1))
    df_encoded['year_cos'] = np.cos(2 * np.pi * (df_encoded['year'] - df_encoded['year'].min()) / 
                                    (df_encoded['year'].max() - df_encoded['year'].min() + 1))

    # Suppression des colonnes 'month' et 'year' après transformation
    df_encoded = df_encoded.drop(columns=['month', 'year'])

    return df_encoded

# Calculer la matrice de corrélation et afficher la heatmap
corr_matrix = encode_and_transform(past_data).corr()
plt.figure(figsize=(40, 40))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Heatmap des Corrélations')
plt.show();

# Suite à cette étude on a décidé de supprimer la variable 'Rebel Groups' car corrélation à 1 avec 'terrorist events'
# On a également supprimé la variable 'total_events' car corrlélée à 98% à 'Rebel Groups', 'terrorist events' et 'Civilians'

In [ ]:
past_data.info()

In [ ]:
# autre visualisation des variables 2 à deux par nuages de points
plt.figure(figsize=(40, 40))
liste_colonnes = ['admin1','year','month', 'terrorist_events', 'fatalities', 'Civilians', 'External/Other Forces', 'Identity Militias','Political Militias', 'State Forces',
                  'Battles', 'Explosions/Remote violence', 'Strategic developments', 'Violence against civilians']
sns.pairplot(past_data[liste_colonnes])
plt.title('Nuages d epoints des variables 2 à 2')
plt.show()

# on voit qu'aucune variable explicative ne suit de loi normale
# une partie des variables semblent avoir des relations linéaires entre elles


In [ ]:
## TEST MODELE RANDOM FOREST REGRESSOR
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)

# On instancie un modèle Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# On entraîne notre modèle
rf.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_rf = rf.predict(X_test_encoded)

# Évaluation du modèle
r2_rf = rf.score(X_test_encoded, y_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
pearson_corr_rf, _ = pearsonr(y_test, y_pred_rf)

print("Random Forest R2:", r2_rf)
print("Random Forest RMSE:", rmse_rf)
print("Random Forest MAE:", mae_rf)
print("Random Forest Pearson Correlation:", pearson_corr_rf)



In [ ]:
## TEST MODELE RANDOM FOREST REGRESSOR
## AVEC GRIDSEARCH
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)

# On définit la grille d'hyperparamètres
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10, 20, 50],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': ['auto', 'sqrt', 'log2']
}

# On instancie le modèle Random Forest
rf = RandomForestRegressor(random_state=42)

# On instancie le GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, scoring='r2', n_jobs=-1)

# On entraîne le GridSearchCV
grid_search.fit(X_train_encoded, y_train)

# Meilleurs paramètres trouvés par GridSearchCV
best_params = grid_search.best_params_
print("Meilleurs paramètres du GridSearchCV:", best_params)

# Prédictions sur l'ensemble de test
y_pred_rf = grid_search.predict(X_test_encoded)

# Évaluation du modèle
r2_rf = grid_search.score(X_test_encoded, y_test)
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
mae_rf = mean_absolute_error(y_test, y_pred_rf)
pearson_corr_rf, _ = pearsonr(y_test, y_pred_rf)

print("Random Forest Gridsearch R2:", r2_rf)
print("Random Forest Gridsearch RMSE:", rmse_rf)
print("Random Forest Gridsearch MAE:", mae_rf)
print("Random Forest Gridsearch Pearson Correlation:", pearson_corr_rf)

In [ ]:
## TEST MODELE RANDOM FOREST REGRESSOR
## AVEC REDUCTION DE DIMENSION (PCA, LDA, UMAP)
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)

##
# PCA
pca = PCA()
#pca = PCA(n_components = 0.9)
X_train_pca = pca.fit_transform(X_train_encoded)
X_test_pca = pca.transform(X_test_encoded)

# On entraine un modèle RF avec la PCA
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_train_pca, y_train)

# Prédictions de la PCA
y_pred_pca = rf.predict(X_test_pca)

# Évaluation de la PCA sur les données historiques
r2_pca = rf.score(X_test_pca, y_test)
rmse_pca = np.sqrt(mean_squared_error(y_test, y_pred_pca))
mae_pca = mean_absolute_error(y_test, y_pred_pca)
pearson_corr_pca, _ = pearsonr(y_test, y_pred_pca)

print("Random Forest PCA R2:", r2_pca)
print("Random Forest PCA RMSE:", rmse_pca)
print("Random Forest PCA MAE:", mae_pca)
print("Random Forest PCA Pearson Correlation:", pearson_corr_pca)


##
# LDA
lda = LDA()
X_train_lda = lda.fit_transform(X_train_encoded, y_train)
X_test_lda = lda.transform(X_test_encoded)

# On entraine un modèle RF avec la LDA
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_train_lda, y_train)

# Prédictions de la LDA
y_pred_lda = rf.predict(X_test_lda)

# Évaluation de la LDA sur les données historiques
r2_lda = rf.score(X_test_lda, y_test)
rmse_lda = np.sqrt(mean_squared_error(y_test, y_pred_lda))
mae_lda = mean_absolute_error(y_test, y_pred_lda)
pearson_corr_lda, _ = pearsonr(y_test, y_pred_lda)

print("\nRandom Forest LDA R2:", r2_lda)
print("Random Forest LDA RMSE:", rmse_lda)
print("Random Forest LDA MAE:", mae_lda)
print("Random Forest LDA Pearson Correlation:", pearson_corr_lda)


##
# UMAP

# On cherche à l'aide d'un GridSearch les meilleurs paramètres
params = {
    'n_neighbors': [5, 10, 15],
    'min_dist': [0.1, 0.5, 0.9],
    'n_components': [2, 3, 4]
}

# On effectue une validation croisée pour évaluer les performances pour chaque combinaison de paramètres
umap_cv = UMAP()
grid_search = GridSearchCV(estimator=umap_cv, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

# On entraine le modèle avec le meilleur hyperparamètre
best_umap_model = grid_search.best_estimator_
X_train_umap = best_umap_model.fit_transform(X_train_encoded)
X_test_umap = best_umap_model.transform(X_test_encoded)

# On entraine un modèle RF sur les données réduites avec UMAP
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(n_estimators=100, random_state=42)

rf.fit(X_train_umap, y_train)

# Prédictions du modèle sur les données réduites avec UMAP
y_pred_umap = rf.predict(X_test_umap)

# Évaluation de UMAP sur les données historiques
r2_umap = rf.score(X_test_umap, y_test)
rmse_umap = np.sqrt(mean_squared_error(y_test, y_pred_umap))
mae_umap = mean_absolute_error(y_test, y_pred_umap)
pearson_corr_umap, _ = pearsonr(y_test, y_pred_umap)

print("\nRandom Forest UMAP R2:", r2_umap)
print("Random Forest UMAP RMSE:", rmse_umap)
print("Random Forest UMAP MAE:", mae_umap)
print("Random Forest UMAP Pearson Correlation:", pearson_corr_umap)


In [ ]:
## TEST MODELE RANDOM FOREST REGRESSOR
## AVEC REDUCTION DE DIMENSION (PCA, LDA, UMAP)
## AVEC GRIDSEARCH
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)


##
# PCA
pca = PCA()
#pca = PCA(n_components = 0.9)
X_train_pca = pca.fit_transform(X_train_encoded)
X_test_pca = pca.transform(X_test_encoded)

# On entraine un modèle RF avec la PCA
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_leaf=1,
                           min_samples_split=2, n_estimators=500, random_state=42)

rf.fit(X_train_pca, y_train)

# Prédictions de la PCA
y_pred_pca = rf.predict(X_test_pca)

# Évaluation de la PCA sur les données historiques
r2_pca = rf.score(X_test_pca, y_test)
rmse_pca = np.sqrt(mean_squared_error(y_test, y_pred_pca))
mae_pca = mean_absolute_error(y_test, y_pred_pca)
pearson_corr_pca, _ = pearsonr(y_test, y_pred_pca)

print("Random Forest PCA Gridsearch R2:", r2_pca)
print("Random Forest PCA Gridsearch RMSE:", rmse_pca)
print("Random Forest PCA Gridsearch MAE:", mae_pca)
print("Random Forest PCA Gridsearch Pearson Correlation:", pearson_corr_pca)


##
# LDA
lda = LDA()
X_train_lda = lda.fit_transform(X_train_encoded, y_train)
X_test_lda = lda.transform(X_test_encoded)

# On entraine un modèle RF avec la LDA
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_leaf=1,
                           min_samples_split=2, n_estimators=500, random_state=42)

rf.fit(X_train_lda, y_train)

# Prédictions de la LDA
y_pred_lda = rf.predict(X_test_lda)

# Évaluation de la LDA sur les données historiques
r2_lda = rf.score(X_test_lda, y_test)
rmse_lda = np.sqrt(mean_squared_error(y_test, y_pred_lda))
mae_lda = mean_absolute_error(y_test, y_pred_lda)
pearson_corr_lda, _ = pearsonr(y_test, y_pred_lda)

print("\nRandom Forest LDA Gridsearch R2:", r2_lda)
print("Random Forest LDA Gridsearch RMSE:", rmse_lda)
print("Random Forest LDA Gridsearch MAE:", mae_lda)
print("Random Forest LDA Gridsearch Pearson Correlation:", pearson_corr_lda)


##
# UMAP

# On cherche à l'aide d'un GridSearch les meilleurs paramètres
params = {
    'n_neighbors': [5, 10, 15],
    'min_dist': [0.1, 0.5, 0.9],
    'n_components': [2, 3, 4]
}

# On effectue une validation croisée pour évaluer les performances pour chaque combinaison de paramètres
umap_cv = UMAP()
grid_search = GridSearchCV(estimator=umap_cv, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

# On entraine le modèle avec le meilleur hyperparamètre
best_umap_model = grid_search.best_estimator_
X_train_umap = best_umap_model.fit_transform(X_train_encoded)
X_test_umap = best_umap_model.transform(X_test_encoded)

# On entraine un modèle RF sur les données réduites avec UMAP
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_leaf=1,
                           min_samples_split=2, n_estimators=500, random_state=42)

rf.fit(X_train_umap, y_train)

# Prédictions du modèle sur les données réduites avec UMAP
y_pred_umap = rf.predict(X_test_umap)

# Évaluation de UMAP sur les données historiques
r2_umap = rf.score(X_test_umap, y_test)
rmse_umap = np.sqrt(mean_squared_error(y_test, y_pred_umap))
mae_umap = mean_absolute_error(y_test, y_pred_umap)
pearson_corr_umap, _ = pearsonr(y_test, y_pred_umap)

print("\nRandom Forest UMAP Gridsearch R2:", r2_umap)
print("Random Forest UMAP Gridsearch RMSE:", rmse_umap)
print("Random Forest UMAP Gridsearch MAE:", mae_umap)
print("Random Forest UMAP Gridsearch Pearson Correlation:", pearson_corr_umap)

In [ ]:
## TEST MODELE RANDOM FOREST REGRESSOR
## AVEC BOOSTING ET BAGGING
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)


##
# Boosting

# On instancie un modèle Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# On entraîne notre modèle
rf.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_rf = rf.predict(X_test_encoded)

ar = AdaBoostRegressor(estimator=rf, n_estimators=400)
ar.fit(X_train_encoded, y_train)

y_pred_ar = ar.predict(X_test_encoded)

# Évaluation du modèle
r2_ar = ar.score(X_test_encoded, y_test)
rmse_ar = np.sqrt(mean_squared_error(y_test, y_pred_ar))
mae_ar = mean_absolute_error(y_test, y_pred_ar)
pearson_corr_ar, _ = pearsonr(y_test, y_pred_ar)

print("Random Forest Boosting R2:", r2_ar)
print("Random Forest Boosting RMSE:", rmse_ar)
print("Random Forest Boosting MAE:", mae_ar)
print("Random Forest Boosting Pearson Correlation:", pearson_corr_ar)


##
# Bagging

# On instancie un modèle Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# On entraîne notre modèle
rf.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_rf = rf.predict(X_test_encoded)

br = BaggingRegressor(n_estimators=1000, oob_score=True)
br.fit(X_train_encoded, y_train)

y_pred_br = br.predict(X_test_encoded)

# Évaluation du modèle
r2_br = br.score(X_test_encoded, y_test)
rmse_br = np.sqrt(mean_squared_error(y_test, y_pred_br))
mae_br = mean_absolute_error(y_test, y_pred_br)
pearson_corr_br, _ = pearsonr(y_test, y_pred_br)

print("\nRandom Forest Bagging R2:", r2_br)
print("Random Forest Bagging RMSE:", rmse_br)
print("Random Forest Bagging MAE:", mae_br)
print("Random Forest Bagging Pearson Correlation:", pearson_corr_br)

In [ ]:
## TEST MODELE RANDOM FOREST REGRESSOR
## AVEC BOOSTING ET BAGGING
## AVEC GRIDSEARCH
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)


##
# Boosting

# On instancie un modèle Random Forest
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_leaf=1,
                           min_samples_split=2, n_estimators=500, random_state=42)

# On entraîne notre modèle
rf.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_rf = rf.predict(X_test_encoded)

ar = AdaBoostRegressor(estimator=rf, n_estimators=400)
ar.fit(X_train_encoded, y_train)

y_pred_ar = ar.predict(X_test_encoded)

# Évaluation du modèle
r2_ar = ar.score(X_test_encoded, y_test)
rmse_ar = np.sqrt(mean_squared_error(y_test, y_pred_ar))
mae_ar = mean_absolute_error(y_test, y_pred_ar)
pearson_corr_ar, _ = pearsonr(y_test, y_pred_ar)

print("Random Forest Boosting Gridsearch R2:", r2_ar)
print("Random Forest Boosting Gridsearch RMSE:", rmse_ar)
print("Random Forest Boosting Gridsearch MAE:", mae_ar)
print("Random Forest Boosting Gridsearch Pearson Correlation:", pearson_corr_ar)


##
# Bagging

# On instancie un modèle Random Forest
# avec les meilleurs paramètres du Gridsearch
rf = RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_leaf=1,
                           min_samples_split=2, n_estimators=500, random_state=42)

# On entraîne notre modèle
rf.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_rf = rf.predict(X_test_encoded)

br = BaggingRegressor(n_estimators=1000, oob_score=True)
br.fit(X_train_encoded, y_train)

y_pred_br = br.predict(X_test_encoded)

# Évaluation du modèle
r2_br = br.score(X_test_encoded, y_test)
rmse_br = np.sqrt(mean_squared_error(y_test, y_pred_br))
mae_br = mean_absolute_error(y_test, y_pred_br)
pearson_corr_br, _ = pearsonr(y_test, y_pred_br)

print("\nRandom Forest Bagging Gridsearch R2:", r2_br)
print("Random Forest Bagging Gridsearch RMSE:", rmse_br)
print("Random Forest Bagging Gridsearch MAE:", mae_br)
print("Random Forest Bagging Gridsearch Pearson Correlation:", pearson_corr_br)

In [ ]:
## TEST MODELE XGBOOST REGRESSOR
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)

# On instancie un modèle XGBoost
xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', random_state=42)

# On entraîne notre modèle
xgb_model.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test_encoded)

# Évaluation du modèle
r2_xgb = xgb_model.score(X_test_encoded, y_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
pearson_corr_xgb, _ = pearsonr(y_test, y_pred_xgb)

print("XGBoost R2:", r2_xgb)
print("XGBoost RMSE:", rmse_xgb)
print("XGBoost MAE:", mae_xgb)
print("XGBoost Pearson Correlation:", pearson_corr_xgb)


In [ ]:
## TEST MODELE XGBOOST REGRESSOR
## AVEC GRIDSEARCH
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)

param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 1.5, 2]
}

# On instancie le modèle XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

# On instancie le GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, 
                           cv=5, scoring='r2', n_jobs=-1)

# On entraîne le GridSearchCV
grid_search.fit(X_train_encoded, y_train)

# Meilleurs paramètres trouvés par GridSearchCV
best_params = grid_search.best_params_
print("Best parameters found by GridSearchCV:", best_params)

# Prédictions sur l'ensemble de test
y_pred_xgb = grid_search.predict(X_test_encoded)

# Évaluation du modèle
r2_xgb = grid_search.score(X_test_encoded, y_test)
rmse_xgb = np.sqrt(mean_squared_error(y_test, y_pred_xgb))
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
pearson_corr_xgb, _ = pearsonr(y_test, y_pred_xgb)

print("XGBoost Gridsearch R2:", r2_xgb)
print("XGBoost Gridsearch RMSE:", rmse_xgb)
print("XGBoost Gridsearch MAE:", mae_xgb)
print("XGBoost Gridsearch Pearson Correlation:", pearson_corr_xgb)


In [ ]:
## TEST MODELE XGBOOST REGRESSOR
## AVEC REDUCTION DE DIMENSION (PCA, LDA, UMAP)
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)


##
# PCA

pca = PCA()
#pca = PCA(n_components = 0.9)
X_train_pca = pca.fit_transform(X_train_encoded)
X_test_pca = pca.transform(X_test_encoded)

# On entraine un modèle Xgboost avec la PCA
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_pca, y_train)

# Prédictions de la PCA
y_pred_pca = xgb_model.predict(X_test_pca)

# Évaluation de la PCA sur les données historiques
r2_pca = xgb_model.score(X_test_pca, y_test)
rmse_pca = np.sqrt(mean_squared_error(y_test, y_pred_pca))
mae_pca = mean_absolute_error(y_test, y_pred_pca)
pearson_corr_pca, _ = pearsonr(y_test, y_pred_pca)

print("XGBoost PCA R2:", r2_pca)
print("XGBoost PCA RMSE:", rmse_pca)
print("XGBoost PCA MAE:", mae_pca)
print("XGBoost PCA Pearson Correlation:", pearson_corr_pca)


##
# LDA

lda = LDA()
X_train_lda = lda.fit_transform(X_train_encoded, y_train)
X_test_lda = lda.transform(X_test_encoded)

# On entraine un modèle Xgboost avec la LDA
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_lda, y_train)

# Prédictions de la LDA
y_pred_lda = xgb_model.predict(X_test_lda)

# Évaluation de la LDA sur les données historiques
r2_lda = xgb_model.score(X_test_lda, y_test)
rmse_lda = np.sqrt(mean_squared_error(y_test, y_pred_lda))
mae_lda = mean_absolute_error(y_test, y_pred_lda)
pearson_corr_lda, _ = pearsonr(y_test, y_pred_lda)

print("\nXGBoost LDA R2:", r2_lda)
print("XGBoost LDA RMSE:", rmse_lda)
print("XGBoost LDA MAE:", mae_lda)
print("XGBoost LDA Pearson Correlation:", pearson_corr_lda)


##
# UMAP

# On cherche à l'aide d'un GridSearch les meilleurs paramètres
params = {
    'n_neighbors': [5, 10, 15],
    'min_dist': [0.1, 0.5, 0.9],
    'n_components': [2, 3, 4]
}

# On effectue une validation croisée pour évaluer les performances pour chaque combinaison de paramètres
umap_cv = UMAP()
grid_search = GridSearchCV(estimator=umap_cv, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

# On entraine le modèle avec le meilleur hyperparamètre
best_umap_model = grid_search.best_estimator_
X_train_umap = best_umap_model.fit_transform(X_train_encoded)
X_test_umap = best_umap_model.transform(X_test_encoded)

# On entraine un modèle Xgboost avec UMAP
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_umap, y_train)

# Prédictions du modèle sur les données réduites avec UMAP
y_pred_umap = xgb_model.predict(X_test_umap)

# Évaluation de UMAP sur les données historiques
r2_umap = xgb_model.score(X_test_umap, y_test)
rmse_umap = np.sqrt(mean_squared_error(y_test, y_pred_umap))
mae_umap = mean_absolute_error(y_test, y_pred_umap)
pearson_corr_umap, _ = pearsonr(y_test, y_pred_umap)

print("\nXGBoost UMAP R2:", r2_umap)
print("XGBoost UMAP RMSE:", rmse_umap)
print("XGBoost UMAP MAE:", mae_umap)
print("XGBoost UMAP Pearson Correlation:", pearson_corr_umap)


In [ ]:
## TEST MODELE XGBOOST REGRESSOR
## AVEC REDUCTION DE DIMENSION (PCA, LDA, UMAP)
## AVEC GRIDSEARCH
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)


##
# PCA

pca = PCA()
#pca = PCA(n_components = 0.9)
X_train_pca = pca.fit_transform(X_train_encoded)
X_test_pca = pca.transform(X_test_encoded)

# On entraine un modèle Xgboost avec la PCA
# avec les meilleurs paramètres du Gridsearch
xgb_model = xgb.XGBRegressor(colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3,
                             n_estimators= 300, reg_alpha= 0, reg_lambda= 2,
                             subsample= 0.6, objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_pca, y_train)

# Prédictions de la PCA
y_pred_pca = xgb_model.predict(X_test_pca)

# Évaluation de la PCA sur les données historiques
r2_pca = xgb_model.score(X_test_pca, y_test)
rmse_pca = np.sqrt(mean_squared_error(y_test, y_pred_pca))
mae_pca = mean_absolute_error(y_test, y_pred_pca)
pearson_corr_pca, _ = pearsonr(y_test, y_pred_pca)

print("XGBoost PCA Gridsearch R2:", r2_pca)
print("XGBoost PCA Gridsearch RMSE:", rmse_pca)
print("XGBoost PCA Gridsearch MAE:", mae_pca)
print("XGBoost PCA Gridsearch Pearson Correlation:", pearson_corr_pca)


##
# LDA

lda = LDA()
X_train_lda = lda.fit_transform(X_train_encoded, y_train)
X_test_lda = lda.transform(X_test_encoded)

# On entraine un modèle Xgboost avec la LDA
# avec les meilleurs paramètres du Gridsearch
xgb_model = xgb.XGBRegressor(colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3,
                             n_estimators= 300, reg_alpha= 0, reg_lambda= 2,
                             subsample= 0.6, objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_lda, y_train)

# Prédictions de la LDA
y_pred_lda = xgb_model.predict(X_test_lda)

# Évaluation de la LDA sur les données historiques
r2_lda = xgb_model.score(X_test_lda, y_test)
rmse_lda = np.sqrt(mean_squared_error(y_test, y_pred_lda))
mae_lda = mean_absolute_error(y_test, y_pred_lda)
pearson_corr_lda, _ = pearsonr(y_test, y_pred_lda)

print("\nXGBoost LDA Gridsearch R2:", r2_lda)
print("XGBoost LDA Gridsearch RMSE:", rmse_lda)
print("XGBoost LDA Gridsearch MAE:", mae_lda)
print("XGBoost LDA Gridsearch Pearson Correlation:", pearson_corr_lda)


##
# UMAP

# On cherche à l'aide d'un GridSearch les meilleurs paramètres
params = {
    'n_neighbors': [5, 10, 15],
    'min_dist': [0.1, 0.5, 0.9],
    'n_components': [2, 3, 4]
}

# On effectue une validation croisée pour évaluer les performances pour chaque combinaison de paramètres
umap_cv = UMAP()
grid_search = GridSearchCV(estimator=umap_cv, param_grid=params, cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_encoded, y_train)

# On entraine le modèle avec le meilleur hyperparamètre
best_umap_model = grid_search.best_estimator_
X_train_umap = best_umap_model.fit_transform(X_train_encoded)
X_test_umap = best_umap_model.transform(X_test_encoded)

# On entraine un modèle Xgboost avec UMAP
# avec les meilleurs paramètres du Gridsearch
xgb_model = xgb.XGBRegressor(colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3,
                             n_estimators= 300, reg_alpha= 0, reg_lambda= 2,
                             subsample= 0.6, objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_umap, y_train)

# Prédictions du modèle sur les données réduites avec UMAP
y_pred_umap = xgb_model.predict(X_test_umap)

# Évaluation de UMAP sur les données historiques
r2_umap = xgb_model.score(X_test_umap, y_test)
rmse_umap = np.sqrt(mean_squared_error(y_test, y_pred_umap))
mae_umap = mean_absolute_error(y_test, y_pred_umap)
pearson_corr_umap, _ = pearsonr(y_test, y_pred_umap)

print("\nXGBoost UMAP Gridsearch R2:", r2_umap)
print("XGBoost UMAP Gridsearch RMSE:", rmse_umap)
print("XGBoost UMAP Gridsearch MAE:", mae_umap)
print("XGBoost UMAP Gridsearch Pearson Correlation:", pearson_corr_umap)


In [ ]:
## TEST MODELE XGBOOST
## AVEC BOOSTING ET BAGGING
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)


##
# Boosting

# On entraine un modèle Xgboost avec UMAP
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test_encoded)

ar = AdaBoostRegressor(estimator=xgb_model, n_estimators=400)

ar.fit(X_train_encoded, y_train)

y_pred_ar = ar.predict(X_test_encoded)

# Évaluation du modèle
r2_ar = ar.score(X_test_encoded, y_test)
rmse_ar = np.sqrt(mean_squared_error(y_test, y_pred_ar))
mae_ar = mean_absolute_error(y_test, y_pred_ar)
pearson_corr_ar, _ = pearsonr(y_test, y_pred_ar)

print("XGBoost Boosting R2:", r2_ar)
print("XGBoost Boosting RMSE:", rmse_ar)
print("XGBoost Boosting MAE:", mae_ar)
print("XGBoost Boosting Pearson Correlation:", pearson_corr_ar)


##
# Bagging

# On entraine un modèle Xgboost avec UMAP
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test_encoded)

br = BaggingRegressor(n_estimators=1000, oob_score=True)

br.fit(X_train_encoded, y_train)

y_pred_br = br.predict(X_test_encoded)

# Évaluation du modèle
r2_br = br.score(X_test_encoded, y_test)
rmse_br = np.sqrt(mean_squared_error(y_test, y_pred_br))
mae_br = mean_absolute_error(y_test, y_pred_br)
pearson_corr_br, _ = pearsonr(y_test, y_pred_br)

print("\nXGBoost Bagging R2:", r2_br)
print("XGBoost Bagging RMSE:", rmse_br)
print("XGBoost Bagging MAE:", mae_br)
print("XGBoost Bagging Pearson Correlation:", pearson_corr_br)

In [ ]:
## TEST MODELE XGBOOST
## AVEC BOOSTING ET BAGGING
## AVEC GRIDSEARCH
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)


##
# Boosting

# On entraine un modèle Xgboost avec UMAP
# avec les meilleurs paramètres du Gridsearch
xgb_model = xgb.XGBRegressor(colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3,
                             n_estimators= 300, reg_alpha= 0, reg_lambda= 2,
                             subsample= 0.6, objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test_encoded)

ar = AdaBoostRegressor(estimator=xgb_model, n_estimators=400)

ar.fit(X_train_encoded, y_train)

y_pred_ar = ar.predict(X_test_encoded)

# Évaluation du modèle
r2_ar = ar.score(X_test_encoded, y_test)
rmse_ar = np.sqrt(mean_squared_error(y_test, y_pred_ar))
mae_ar = mean_absolute_error(y_test, y_pred_ar)
pearson_corr_ar, _ = pearsonr(y_test, y_pred_ar)

print("XGBoost Boosting Gridsearch R2:", r2_ar)
print("XGBoost Boosting Gridsearch RMSE:", rmse_ar)
print("XGBoost Boosting Gridsearch MAE:", mae_ar)
print("XGBoost Boosting Gridsearch Pearson Correlation:", pearson_corr_ar)


##
# Bagging

# On entraine un modèle Xgboost avec UMAP
# avec les meilleurs paramètres du Gridsearch
xgb_model = xgb.XGBRegressor(colsample_bytree= 1.0, learning_rate= 0.1, max_depth= 3,
                             n_estimators= 300, reg_alpha= 0, reg_lambda= 2,
                             subsample= 0.6, objective='reg:squarederror', random_state=42)

xgb_model.fit(X_train_encoded, y_train)

# Prédictions sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test_encoded)

br = BaggingRegressor(n_estimators=1000, oob_score=True)

br.fit(X_train_encoded, y_train)

y_pred_br = br.predict(X_test_encoded)

# Évaluation du modèle
r2_br = br.score(X_test_encoded, y_test)
rmse_br = np.sqrt(mean_squared_error(y_test, y_pred_br))
mae_br = mean_absolute_error(y_test, y_pred_br)
pearson_corr_br, _ = pearsonr(y_test, y_pred_br)

print("\nXGBoost Bagging Gridsearch R2:", r2_br)
print("XGBoost Bagging Gridsearch RMSE:", rmse_br)
print("XGBoost Bagging Gridsearch MAE:", mae_br)
print("XGBoost Bagging Gridsearch Pearson Correlation:", pearson_corr_br)

In [ ]:
## TEST MODELE SVM REGRESSOR
############################################################

# On récupère nos jeux d'entrainement et de test
X_train_encoded, X_test_encoded, _, y_train, y_test, _ = past_data_transformation(past_data)

# On instancie un modèle SVR
svr_model = svm.SVR()
parametres = {'C':[1,10, 50, 100, 1000], 'kernel':['rbf', 'poly', 'sigmoid'], 'gamma':[0.001, 0.01, 0.1]}
grid_svr_model = GridSearchCV(estimator=svr_model, param_grid=parametres, scoring = 'neg_mean_absolute_error')

# On entraine le modéle
grille = grid_svr_model.fit(X_train_encoded,y_train)
print(pd.DataFrame.from_dict(grille.cv_results_).loc[:,['params', 'mean_test_score']]) 
print(grid_svr_model.best_params_)

#Prédiction sur l'ensemble de test
y_pred_svr = grid_svr_model.predict(X_test_encoded)

# Évaluation du modèle
r2_svr_train = grid_svr_model.score(X_train_encoded, y_train)
r2_svr_test = grid_svr_model.score(X_test_encoded, y_test)
rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))
mae_svr = mean_absolute_error(y_test, y_pred_svr)
mape_svr = mean_absolute_percentage_error(y_test, y_pred_svr)
pearson_corr_svr, _ = pearsonr(y_test, y_pred_svr)

print("SVR R2 train:", r2_svr_train)
print("SVR R2 test:", r2_svr_test)
print("SVR RMSE:", rmse_svr)
print("SVR MAE:", mae_svr)
print("SVR MAPE:", mape_svr)
print("Pearson Correlation (SVR):", pearson_corr_svr)